In [26]:
print('jmd shree ganesha')

jmd shree ganesha


In [27]:
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.vectorstores import chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import os


In [28]:
dataset = r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\data"

In [29]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [30]:
extracted_data = load_pdf(dataset)

In [31]:
def text_splitter(data_pdf):
    splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=10)
    chunks = splitter.split_documents(data_pdf)
    
    return chunks

In [32]:
text_split_chunks = text_splitter(extracted_data)

In [33]:
text_split_chunks

[Document(page_content='Yes, you can book appointments and many treatments directly with us. For most procedures, a \nGP referral is required.', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf', 'page': 0}),
 Document(page_content='If you are insured, we also recommend that you check with your insurer that you are covered to \nsee your preferred consultant at our hospital.', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf', 'page': 0}),
 Document(page_content='You can easily book an appointment online with many of our consultants. To find a consultant', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf', 'page': 0}),
 Document(page_content='near you, please clic k here. For more information, please call 0330 173 8200.', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf', 'page': 0}),
 Docu

In [34]:
print(len(text_split_chunks))

84


In [35]:
def text_to_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [36]:
embeddings = text_to_embeddings()

c:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\chatbot\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [37]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [38]:
persist_directory = 'chat'

vectordb = chroma.Chroma.from_documents(documents=text_split_chunks, embedding=embeddings, persist_directory=persist_directory)

In [39]:
vectordb.persist()
vectordb = None

In [40]:
vectordb = chroma.Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [41]:
retriever = vectordb.as_retriever()

In [42]:
docsearch = retriever.get_relevant_documents('Do I need a referral from my GP?')

In [43]:
len(docsearch)
print(docsearch)

[Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': '/home/pheonix/Aster/FAQ-Chatbot/data/dataset.pdf'}), Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf'}), Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf'}), Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf'})]


In [44]:
retriever = vectordb.as_retriever(search_kwargs={'k':4})

In [45]:
retriever.search_type

'similarity'

In [46]:
retriever.search_kwargs

{'k': 4}

In [47]:
prompt_template="""
Use the following pieces of information to answer the user's question. You are a FAQ chatbot for Nuffield Health Hostipal.
If you don't know the answer, just say that to contact the helpline number. Not to ask for phone number

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [48]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [49]:
llm=ctransformers.CTransformers(model=r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.7})

In [51]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(search_kwargs={'k': 5}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [52]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Yes, most procedures at Nuffield Health Hospital require a GP referral. This will ensure that you are booked in for the appropriate specialty. There may be some treatments where a referral is not needed, so please check with our helpline for more information.
